In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
pd.set_option('max_colwidth', 800)

In [2]:
results = [file for file in os.listdir('../Data') if 'result' in file]
df_list = []
for result in results:
    df = pd.read_csv('../Data/'+result,na_values=0)
    df['train_type'] = 'base' if 'base' in result else ('start' if 'start' in result else ('2stage' if '2stage' in result else 'org'))
    df['file_name'] = result
    df_list.append(df)
df = pd.concat(df_list,ignore_index=True)#.fillna(1e4)

In [3]:
# find the files that corresponds to min_loss for each type
val_loss_col = ['val_type_'+str(i) for i in range(8)]
row_min = np.nanargmin(df[val_loss_col].values,0)
df_min_loss = df.loc[row_min,val_loss_col+['file_name']]
df_min_trainType = df.loc[row_min,['train_type']]
df_min_loss

,val_type_0,val_type_1,val_type_2,val_type_3,val_type_4,val_type_5,val_type_6,val_type_7,file_name
499,-0.666885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,results_False_<class 'functions_refactor.NNConv_block'>_<class 'functions_refactor.cat3Head_type'>__data_ACSF_expand.pickle_32_128_2_2_2_2_25_type_0_start.csv
53,NaN,-1.242345,NaN,NaN,NaN,NaN,NaN,NaN,results_False_<class 'functions_refactor.schnet_block'>_<class 'functions_refactor.cat3Head_type'>__data_ACSF_expand.pickle_32_128_2_2_2_2_15_type_1_2stage.csv
174,NaN,NaN,-1.637411,NaN,NaN,NaN,NaN,NaN,results_False_<class 'functions_refactor.NNConv_block'>_<class 'functions_refactor.cat3Head_type'>__data_ACSF_expand.pickle_32_128_2_2_2_2_25_type_2_start.csv
655,-0.174251,-0.805135,-1.451124,-2.398752,-2.297072,-1.177319,-2.064010,-2.484622,results_False_<class 'functions_refactor.MEGNet_block'>_None__data_ACSF_expand.pickle_32_128_0.4_3_3_2_10_base.csv
661,0.310971,-1.119503,-1.417348,-2.192395,-2.339651,-1.154478,-2.149081,-2.444351,results_False_<class 'functions_refactor.MEGNet_block'>_None__data_ACSF_expand.pickle_32_128_0.4_3_3_2_10_base.csv
79,NaN,NaN,NaN,NaN,NaN,-1.235491,NaN,NaN,results_False_<class 'functions_refactor.NNConv_block'>_<class 'functions_refactor.cat3Head_type'>__data_ACSF_expand.pickle_32_128_2_2_2_2_25_type_5_start.csv
661,0.310971,-1.119503,-1.417348,-2.192395,-2.339651,-1.154478,-2.149081,-2.444351,results_False_<class 'functions_refactor.MEGNet_block'>_None__data_ACSF_expand.pickle_32_128_0.4_3_3_2_10_base.csv
655,-0.174251,-0.805135,-1.451124,-2.398752,-2.297072,-1.177319,-2.064010,-2.484622,results_False_<class 'functions_refactor.MEGNet_block'>_None__data_ACSF_expand.pickle_32_128_0.4_3_3_2_10_base.csv


In [4]:
# load prediction from submission files that corresponds to the best model found in results 
submission = pd.read_csv('../Data/sample_submission.csv')
submission_files = os.listdir('../Submission')
for f,train_type in zip(df_min_loss.file_name.tolist(),df_min_trainType.train_type.tolist()):
    # find right submission file
    submit_file = [sub for sub in submission_files if (train_type in sub) & ('all_types' in sub) &\
                                           (f[8:-20] in sub)]
    assert len(submit_file)==1,'should have one submission file, but got {}'.format(len(submit_file))       
    submit_df = pd.read_csv('../Submission/'+submit_file[0])
    
    # get right type prediction from submission file
    temp_type = 'type_'+f[-(6+len(train_type))]
    submission[temp_type] = submit_df[temp_type]

# save results
submission['scalar_coupling_constant'] = submission.iloc[:,2:].mean(1)
submission = submission[['id','scalar_coupling_constant']]
submission.to_csv('../Submission/combine_type.csv',index=False)